# DysonianLineCNN - Нейронна мережа для передбачення параметрів лінії Дайсона

Цей notebook містить код для навчання CNN моделі для передбачення параметрів лінії Дайсона.

## Структура проекту:
- `colab_gpu_check.py` - перевірка системних ресурсів
- `data_loader.py` - завантаження та підготовка даних
- `model.py` - архітектура нейронної мережі
- `trainer.py` - навчання моделі
- `utils.py` - допоміжні функції

In [ ]:
# ⚡️ Клонування репозиторію для запуску в Colab
!git clone https://github.com/AndriiUriadov/DysonLineParamsExtractorCNN.git
%cd DysonLineParamsExtractorCNN

## 1. Перевірка середовища виконання

In [ ]:
# Виконуємо код перевірки середовища виконання
import subprocess
import sys

# Виконуємо файл як модуль
exec(open('colab_gpu_check.py').read())


## 2. Завантаження та підготовка даних

In [ ]:
# Імпортуємо функції для роботи з даними
from data_loader import load_and_preprocess

# Завантажуємо та підготовляємо дані
(X_train, X_val, X_test, y_train, y_val, y_test, 
 y_train_dict, y_val_dict, y_test_dict, scaler_y, y_min, y_max) = load_and_preprocess()

## 3. Створення моделі

In [ ]:
# Імпортуємо функції для створення моделі
from model import create_model, test_model
from utils import check_device_availability

# Перевіряємо доступність GPU
device = check_device_availability()

# Створюємо модель
model = create_model(device=device)

# Тестуємо модель
test_model(model)

## 4. Навчання моделі

In [ ]:
# Імпортуємо функції для навчання
from trainer import create_trainer

# Оптимізації пам'яті GPU
import torch
torch.cuda.empty_cache()
torch.backends.cudnn.benchmark = True

# Створюємо тренер
trainer = create_trainer(model, device=device)

# Створюємо DataLoader з меншим batch_size
train_loader, val_loader, test_loader = trainer.create_data_loaders(
    X_train, X_val, X_test, y_train, y_val, y_test, batch_size=8
)

# Навчаємо модель
history = trainer.train(train_loader, val_loader, num_epochs=100)


## 5. Оцінка моделі

In [ ]:
# Оцінюємо модель на тестових даних
test_metrics = trainer.evaluate(test_loader)

# Візуалізуємо історію навчання
trainer.plot_training_history()

## 6. Детальний аналіз результатів

In [ ]:
# Імпортуємо функції для аналізу
from utils import (
    plot_predictions_vs_targets, plot_residuals, plot_correlation_matrix,
    create_prediction_report, save_scaler, save_training_results
)

# Отримуємо передбачення на тестових даних
trainer.model.eval()
with torch.no_grad():
    test_predictions = trainer.model(torch.FloatTensor(X_test).to(device)).cpu().numpy()

# Денормалізуємо передбачення
test_predictions_denorm = denormalize_predictions(test_predictions, scaler_y)
test_targets_denorm = denormalize_predictions(y_test, scaler_y)

# Створюємо детальний звіт
report = create_prediction_report(test_predictions_denorm, test_targets_denorm)

# Візуалізуємо результати
plot_predictions_vs_targets(test_predictions_denorm, test_targets_denorm)
plot_residuals(test_predictions_denorm, test_targets_denorm)
plot_correlation_matrix(test_predictions_denorm, test_targets_denorm)

# Зберігаємо результати
save_scaler(scaler_y, 'scaler_y.pkl')
save_training_results(history, 'training_history.pkl')